In [ ]:
pip install requests pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import requests
import json
import pandas as pd
import time

with open('bids-apps.json', 'r') as file:
    data = json.load(file)

apps = data['apps']
apps_df = pd.DataFrame(apps)
apps_df[['description', 'status', 'gh', 'dh', 'ds_type', 'datatype']].head()

In [ ]:
# Request to docker hub to get and update latest updated image tag to each app
for app in apps:
    dh_image = app['dh']
    response = requests.get(f'https://hub.docker.com/v2/repositories/{dh_image}/tags')
    if response.status_code == 200:
        tags = response.json()['results']
        last_updated = None
        # order tags by last_updated and get the latest one
        for tag in tags:
            if last_updated is None or tag['last_updated'] > last_updated:
                last_updated = tag['last_updated']
                latest_tag = tag['name']
        app['latest_version'] = latest_tag
    else:
        app['latest_version'] = 'Error fetching tags'
    # Sleep to avoid rate limiting of docker hub
    time.sleep(0.5)

# Save the updated json file
with open('bids-apps.json', 'w') as file:
    json.dump(data, file, indent=4)